# DeTexD vs Zampieri, Davidson

Evaluate performance of DeTexD baseline model on the following datasets:

- OLID from Zampieri et. al.
- dataset Davidson et. al.
- DeTexD benchmark dataset


In [4]:
import os
import html
import pandas as pd
import torch
from IPython.display import HTML, display
from sklearn.metrics import classification_report
from datasets import load_dataset
from evaluate_detexd_roberta import predict_delicate
from transformers import pipeline

Datasets:
1. Davidson et. al. https://arxiv.org/pdf/1703.04009.pdf
2. Zampieri et. al. https://aclanthology.org/N19-1144.pdf

### Zampieri et. al. (aka OLID)

3) TASKS AND LABELS

(A) Level A: Offensive language identification

- (NOT) Not Offensive - This post does not contain offense or profanity.
- (OFF) Offensive - This post contains offensive language or a targeted (veiled or direct) offense

In our annotation, we label a post as offensive (OFF) if it contains any form of non-acceptable language (profanity) or a targeted offense, which can be veiled or direct. 

(B) Level B: Automatic categorization of offense types

- (TIN) Targeted Insult and Threats - A post containing an insult or threat to an individual, a group, or others (see categories in sub-task C).
- (UNT) Untargeted - A post containing non-targeted profanity and swearing.

Posts containing general profanity are not targeted, but they contain non-acceptable language.

(C) Level C: Offense target identification

- (IND) Individual - The target of the offensive post is an individual: a famous person, a named individual or an unnamed person interacting in the conversation.
- (GRP) Group - The target of the offensive post is a group of people considered as a unity due to the same ethnicity, gender or sexual orientation, political affiliation, religious belief, or something else.
- (OTH) Other – The target of the offensive post does not belong to any of the previous two categories (e.g., an organization, a situation, an event, or an issue)

In [5]:
# NOTE: unfortunately, the OLID dataset is not easily obtainable at original address
# (https://competitions.codalab.org/competitions/20011/results/34108/data), to further
# reproducibility here we use a local copy of the data derived from https://github.com/idontflow/OLID

olid_data_path = 'olid_data.csv'

olid_df = pd.read_csv(olid_data_path)
olid_df.shape, olid_df.columns

((13240, 7),
 Index(['del', 'id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c',
        'cleaned_tweet'],
       dtype='object'))

In [6]:
olid_df.astype('object').describe().transpose()

,count,unique,top,freq
del,13240,13240,0,1
id,13240,13240,86426,1
tweet,13240,13207,@USER Looks Like The Jokes On Liberals Again. ...,6
subtask_a,13240,2,NOT,8840
subtask_b,4400,2,TIN,3876
subtask_c,3876,3,IND,2407
cleaned_tweet,13156,12797,yes,22


In [7]:
olid_df['subtask_a'].value_counts()

subtask_a
NOT    8840
OFF    4400
Name: count, dtype: int64

In [8]:
# normalize text format
# this dataset contains unescaped HTML entities
olid_df['text'] = olid_df['tweet'].apply(html.unescape)

In [9]:
olid_df['offensive'] = olid_df['subtask_a'].apply(lambda x: int(x == 'OFF'))
olid_df['offensive'].value_counts()

offensive
0    8840
1    4400
Name: count, dtype: int64

### Davidson et. al.

- https://github.com/t-davidson/hate-speech-and-offensive-language/tree/master/data

- count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).
- hate_speech = number of CF users who judged the tweet to be hate speech.
- offensive_language = number of CF users who judged the tweet to be offensive.
- neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.
- class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [11]:
# Davidson et. al.
davidson_data_path = 'davidson.csv'

if not os.path.isfile(davidson_data_path):
    !wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.csv -O - \
        > {davidson_data_path}

--2023-07-10 13:26:10--  https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv [following]
--2023-07-10 13:26:11--  https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2546446 (2.4M) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   2.43M  --.-KB/s    in 0.1s    

2023-07-10 13:26:11 (16.2 MB/s) 

In [12]:
davidson_df = pd.read_csv(davidson_data_path)
davidson_df.shape, davidson_df.columns

((24783, 7),
 Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
        'class', 'tweet'],
       dtype='object'))

In [13]:
davidson_df.astype('object').describe().transpose()

,count,unique,top,freq
Unnamed: 0,24783,24783,0,1
count,24783,5,3,22807
hate_speech,24783,8,0,19790
offensive_language,24783,10,3,13383
neither,24783,10,0,18892
class,24783,3,1,19190
tweet,24783,24783,!!! RT @mayasolovely: As a woman you shouldn't...,1


In [14]:
davidson_df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [15]:
# convert davidson predictions to binary

davidson_df['offensive'] = davidson_df['class'].apply(lambda x: int(x == 1))
davidson_df['hate'] = davidson_df['class'].apply(lambda x: int(x == 0))
davidson_df['offensive_or_hate'] = davidson_df['class'].apply(lambda x: int(x <= 1))

davidson_df['offensive_or_hate'].value_counts(), davidson_df['offensive'].value_counts(), davidson_df['hate'].value_counts()

(offensive_or_hate
 1    20620
 0     4163
 Name: count, dtype: int64,
 offensive
 1    19190
 0     5593
 Name: count, dtype: int64,
 hate
 0    23353
 1     1430
 Name: count, dtype: int64)

In [16]:
# normalize text format
# this dataset contains unescaped HTML entities
davidson_df['text'] = davidson_df['tweet'].apply(html.unescape)

### DeTexD dataset

In [17]:
detexd_test_df = load_dataset("grammarly/detexd-benchmark", split='test').to_pandas()
detexd_test_df.shape, detexd_test_df.columns

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Dataset csv downloaded and prepared to /Users/eis/.cache/huggingface/datasets/grammarly___csv/grammarly--detexd-benchmark-83592511fc08ad4e/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


((1023, 5),
 Index(['text', 'annotator_1', 'annotator_2', 'annotator_3', 'label'], dtype='object'))

In [18]:
detexd_test_df.astype('object').describe().transpose()

,count,unique,top,freq
text,1023,1023,"""), as well as other minority interests and gr...",1
annotator_1,1023,6,0,494
annotator_2,1023,6,0,484
annotator_3,1023,6,0,531
label,1023,2,0,687


### Run DeTexD predictions

In [19]:
classifier = pipeline("text-classification", model="grammarly/detexd-roberta-base", device=torch.device('cpu'), batch_size=8)
predict_delicate(classifier, ['I love you', 'I hate you'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


  0%|          | 0/2 [00:00<?, ?it/s]

[False, False]

In [24]:
davidson_df['detexd_pred'] = predict_delicate(classifier, davidson_df['text'].tolist())

  0%|          | 0/24783 [00:00<?, ?it/s]

In [36]:
olid_df['detexd_pred'] = predict_delicate(classifier, olid_df['text'].tolist())

  0%|          | 0/13240 [00:00<?, ?it/s]

In [41]:
detexd_test_df['detexd_pred'] = predict_delicate(classifier, detexd_test_df['text'].tolist())

  0%|          | 0/1023 [00:00<?, ?it/s]

## Evaluate on Davidson dataset

In [26]:
davidson_df['detexd_pred'].value_counts()

detexd_pred
True     15268
False     9515
Name: count, dtype: int64

In [29]:
# create data subsets for separate hate and offensive label evals
davidson_hate_df = davidson_df[davidson_df['class'].apply(lambda x: x == 0 or x == 2)]
davidson_offensive_df = davidson_df[davidson_df['class'].apply(lambda x: x == 1 or x == 2)]

davidson_hate_df['class'].value_counts(), davidson_offensive_df['class'].value_counts()

(class
 2    4163
 0    1430
 Name: count, dtype: int64,
 class
 1    19190
 2     4163
 Name: count, dtype: int64)

In [27]:
def do_eval(texts, y_true, y_pred, fpfn_samples=15):
    return dict(
        eval=classification_report(
            y_true, y_pred,
            output_dict=True
        )['1'],
        fps=(fps := [t for t, y, yh in zip(texts, y_true, y_pred) if y == 0 and yh == 1]),
        fns=(fns := [t for t, y, yh in zip(texts, y_true, y_pred) if y == 1 and yh == 0]),
        fps_sample=pd.Series(fps).sample(n=fpfn_samples, random_state=0),
        fns_sample=pd.Series(fns).sample(n=fpfn_samples, random_state=0),
    )

def show_eval(eval_results):
    display(HTML(pd.DataFrame([
        {
            'Precision': eval_results['eval']['precision'],
            'Recall': eval_results['eval']['recall'],
            'F1-score': eval_results['eval']['f1-score'],
        }
    ]).style.format({
        'Precision': '{:,.1%}'.format,
        'Recall': '{:,.1%}'.format,
        'F1-score': '{:,.1%}'.format,
    }).to_html(index=False)))

    display(HTML(eval_results['fps_sample'].rename('FPs').to_frame().to_html(index=False)))
    display(HTML(eval_results['fns_sample'].rename('FNs').to_frame().to_html(index=False)))


In [31]:
davidson_hate_eval = do_eval(
    davidson_hate_df['text'],
    davidson_hate_df['hate'],
    davidson_hate_df['detexd_pred'],
)

show_eval(davidson_hate_eval)

,Precision,Recall,F1-score
0,60.9%,79.5%,69.0%


FPs
@SenMikeLee just pulled out his pocket Constitution! Bring it man! @tedcruz just mocked McCain's wacko-birds comment! HaHaHa!!!!! #standtall
I hope the trash I talk abt @CNN's poor product doesn't hurt my chances of getting hired as a legal correspondent. I was jk CNN.
"Nothing says teabagger like a millionaire complaining that they are broke, huh #HillaryClinton? #Hillary2016, give me a break, lol. #p2"
RT @LifeNewsHQ: BREAKING: Florida Pro-Life Gov. Rick Scott Defeats Pro-Abortion Challenger Charlie Crist http://t.co/GpR1hQlEN2 #tcot http:…
"RT @p_w_sterne: @GraceBr92073005 ""There's more than one way to skin a monkey"" --Barack ""Bushmeat"" Obama @1Dittlinger @Txwench"
"RT @WayneDupreeShow: Fear mongering by the left is dying. Wake up, who are you going 2 believe Charlie Rangel and Hussein or Mia Love and C…"
RT @Lrihendry: Pregnant S.E. Cupp’s classic response to ‘liberal dem’ who calls her ‘chunky’ – with bonus tweets http://t.co/U8nlYWEfiO via…
"“@PlMPCESS: In the fashion world ""urban"" means ""stolen from black people"" its ghetto on us, and urban on them haha http://t.co/S0XWGfLJ9C” 🙌"
"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Teen | Cutie lesbian teens toy slits http://t.co/ZS05enjjwm"
"""How about we draft all those gung-ho folks and send them to Afghanistan or other war zones so they can show their... http://t.co/sR1zceHx"


FNs
RT @STROCORLEONE @ThoughtsOfRandy @Yummys_World all deze hoes got past demons in em... YOLO nigga lol + Hahahahaha
"it means that im following orders and being subordinate. sue me, ya dyke"
RT @Drizz_LakerKing: Horrible RT @YeIIowbang: Invader Zim was trash
"I'm literally laughing in shock, amazement, joy. In...just everything. In everything! Pop the champagne American queers. We have arrived."
@kieffer_jason here let me say it more clearly since your retarded ass don't understand
You are a jiggaboo...!
@MorbidMermaid do you work today niglet?
"Everyone tells me the exact same thing but I won't listen, coon"
@saraelizabethj4 that's what happens when you date a colored
@Derek_Kirchner7 @tori_mills02 fags😷😷


In [33]:
davidson_offensive_eval = do_eval(
    davidson_offensive_df['text'],
    davidson_offensive_df['offensive'],
    davidson_offensive_df['detexd_pred'],
)

show_eval(davidson_offensive_eval)

,Precision,Recall,F1-score
0,94.8%,69.8%,80.4%


FPs
@SenMikeLee just pulled out his pocket Constitution! Bring it man! @tedcruz just mocked McCain's wacko-birds comment! HaHaHa!!!!! #standtall
I hope the trash I talk abt @CNN's poor product doesn't hurt my chances of getting hired as a legal correspondent. I was jk CNN.
"Nothing says teabagger like a millionaire complaining that they are broke, huh #HillaryClinton? #Hillary2016, give me a break, lol. #p2"
RT @LifeNewsHQ: BREAKING: Florida Pro-Life Gov. Rick Scott Defeats Pro-Abortion Challenger Charlie Crist http://t.co/GpR1hQlEN2 #tcot http:…
"RT @p_w_sterne: @GraceBr92073005 ""There's more than one way to skin a monkey"" --Barack ""Bushmeat"" Obama @1Dittlinger @Txwench"
"RT @WayneDupreeShow: Fear mongering by the left is dying. Wake up, who are you going 2 believe Charlie Rangel and Hussein or Mia Love and C…"
RT @Lrihendry: Pregnant S.E. Cupp’s classic response to ‘liberal dem’ who calls her ‘chunky’ – with bonus tweets http://t.co/U8nlYWEfiO via…
"“@PlMPCESS: In the fashion world ""urban"" means ""stolen from black people"" its ghetto on us, and urban on them haha http://t.co/S0XWGfLJ9C” 🙌"
"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Teen | Cutie lesbian teens toy slits http://t.co/ZS05enjjwm"
"""How about we draft all those gung-ho folks and send them to Afghanistan or other war zones so they can show their... http://t.co/sR1zceHx"


FNs
"RT @Lyndsay_Sue_x2: Ford. F-O-R-D, just a fancy name for a fag mobile"
butt chug 4lokos or nah
This bitch Rihanna is on some dust or something
RT @Coolie_GBE300: I'm about to go on http://t.co/9btG26MlgU & be the devil & catfish some hoes.
@Rams_6 happy birthday bitch!
"I feel bad having this info but at the same time , karmas a bitch,"
“@ChrizzyFive: Trying to run game on me u dumb bitch !!” @ her tho
@espn Stop blowing the spurs pussies #HalftimeSpeech
RT @RWSurferGirl: Toss 'em out of office? Or just throw 'em in the trash? #trashbucketchallenge has politicians on the run at... https://t.…
Same niccas tryna be around me .. The same niccas that used to down me!


In [34]:
davidson_eval = do_eval(
    davidson_df['text'],
    davidson_df['offensive_or_hate'],
    davidson_df['detexd_pred'],
)

show_eval(davidson_eval)

,Precision,Recall,F1-score
0,95.2%,70.5%,81.0%


FPs
@SenMikeLee just pulled out his pocket Constitution! Bring it man! @tedcruz just mocked McCain's wacko-birds comment! HaHaHa!!!!! #standtall
I hope the trash I talk abt @CNN's poor product doesn't hurt my chances of getting hired as a legal correspondent. I was jk CNN.
"Nothing says teabagger like a millionaire complaining that they are broke, huh #HillaryClinton? #Hillary2016, give me a break, lol. #p2"
RT @LifeNewsHQ: BREAKING: Florida Pro-Life Gov. Rick Scott Defeats Pro-Abortion Challenger Charlie Crist http://t.co/GpR1hQlEN2 #tcot http:…
"RT @p_w_sterne: @GraceBr92073005 ""There's more than one way to skin a monkey"" --Barack ""Bushmeat"" Obama @1Dittlinger @Txwench"
"RT @WayneDupreeShow: Fear mongering by the left is dying. Wake up, who are you going 2 believe Charlie Rangel and Hussein or Mia Love and C…"
RT @Lrihendry: Pregnant S.E. Cupp’s classic response to ‘liberal dem’ who calls her ‘chunky’ – with bonus tweets http://t.co/U8nlYWEfiO via…
"“@PlMPCESS: In the fashion world ""urban"" means ""stolen from black people"" its ghetto on us, and urban on them haha http://t.co/S0XWGfLJ9C” 🙌"
"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Teen | Cutie lesbian teens toy slits http://t.co/ZS05enjjwm"
"""How about we draft all those gung-ho folks and send them to Afghanistan or other war zones so they can show their... http://t.co/sR1zceHx"


FNs
@Z_M_G_ thirsty hoes 💁
RT @HoeFact: Don't trust these hoes http://t.co/03SoHP4P41
"niggas need to learn to ride for there homies, not for the crew hoes smfh."
“@NigggaPino: Splash onna bitch Wednesday tomorrow”
Call dat bitch... http://t.co/p50wtGrz8a
RT @KattWilliams: me as a good friend: *supports and encourages ur hoe activities*
RT @TheoMaxximus: I can post a picture of chrus at like 5 in the morning and the little nigga will favorite and retweet that hoe
RT @CheefPolo: Trippin over a rock >> trippin over a bitch
"RT @thecrewfinest: ""@187XO_: Me and ma niggah @thecrewfinest In the Func fucccccccked upppppp 😂""I'm Here On!!!"
@JalenDGAF @CourtniClark 🔴⚫️🔴⚫️🔴⚫️ no pressure hoe


In [35]:
davidson_eval['fps_sample'].to_csv('davidson_detexd_fps.csv')
davidson_eval['fns_sample'].to_csv('davidson_detexd_fns.csv')

## Evaluate Zampieri dataset

In [38]:
olid_df['detexd_pred'].value_counts()

detexd_pred
False    7165
True     6075
Name: count, dtype: int64

In [39]:
olid_eval = do_eval(
    olid_df['text'],
    olid_df['offensive'],
    olid_df['detexd_pred'],
)

show_eval(olid_eval)

,Precision,Recall,F1-score
0,48.1%,66.4%,55.8%


FPs
"#Liberals don't like to be pushed around. #Conservatives aren't supposed to be autocratic, we are."" You know you're winning when they're shouting exceeds their understanding of rules based law. Best way to watch Liberals heads explode is for Cons to be more unpredictable. URL"
@USER @USER Liberals judge you guilty EVEN when proven innocent. #liberalsneedtheirownpsychward
"@USER @USER I am trying to make a point. Many people who hate the liberal and ndp party are conservatives. And all conservatives care about is money. They don't have to ruin the environment and discriminate against others because of money"""""
1. Trump and the Mueller Investigation 2. Climate Change 3. Guns and Gun Control 4. Unions and Workers #SaveSCOTUS #WhatsAtStake URL
URL #MCCASKILL owes back taxes on her private jet! Has it been paid? @USER #MAGA #Kavanaugh #ChristinaBlaseyFord #QANON #ThursdayMotivation
@USER @USER I recognize the one as #antifa based on the board he becomes 🤣
@USER Or Christian conservatives could man up and do whatever it takes to win.
@USER He's going to need a pair of running shoes so he can look for work when he's sees we're not supporting his dumb ass movies anymore. 🤷‍♀️#walkaway
@USER @USER @USER The historic moniker for the Tories is The Stupid Party. American conservatives test out with lower average intelligence than liberals.
@USER @USER With tons of security that he wants us to NOT HAVE! It blows my mind how many gun control ppl have bodyguards With the guns we R not suppose to have !


FNs
@USER As someone who had never played Spiderman 2 as a kid: I want and tried it a year or 2 back and idk what kinda collective nostalgia people are on but those web swinging mechanics are busted as fuck and kinda trash.
@USER A dying sport
@USER @USER @USER @USER Yes. Yes he is!
@USER @USER Excuse me? They got what they had before the hurricane. Expecting tract houses when you had lean to's and expecting honorable people likeTrump with the corruption they continue to vote in?!? Wake up. He is flushing the swamp here and they need to flush their swamp there.
@USER You voted to trigger Article 50. You are complicit.
@USER Do not think so maybe do a little research buddy!
@USER Never had. Let's fuck it up.
@USER @USER @USER Oh boy just wait! YOU ARE IN FOR A SURPRISE! Can't believe ANYONE to be that IGNORANT! LMAO!
@USER Fools No respect
@USER @USER I’m sure the air that he is breathing is also bad.


In [40]:
olid_df[olid_df['text'] == '@USER A dying sport']

,del,id,tweet,subtask_a,subtask_b,subtask_c,cleaned_tweet,text,offensive,detexd_pred
6572,6572,96293,@USER A dying sport,OFF,UNT,NaN,dying sport,@USER A dying sport,1,False


## Evaluate on DeTexD dataset

In [42]:
detexd_test_df['detexd_pred'].value_counts()

detexd_pred
False    700
True     323
Name: count, dtype: int64

In [44]:
detexd_eval = do_eval(
    detexd_test_df['text'],
    detexd_test_df['label'],
    detexd_test_df['detexd_pred'],
)

show_eval(detexd_eval)

,Precision,Recall,F1-score
0,81.4%,78.3%,79.8%


FPs
"Last I had checked he’s not around anymore. And I’m mostly into female preds, but I do have a few male characters I would like to see art of. So I can understand your frustration with the lack of male preds. And yeah Mode mostly does do futa and other really weird shit"
"Sorry to hear that you have anxiety driving. For me, it's my way of freedom. Whenever i feel depressed or need space, i can just jump in my car and drive to get away from it all. Also, i can do whatever i need to do, go shopping, go to the park, beach, etc. Im in California and it's always congested but freedom means much much more to me. i hope you can find a way to get over your anxiety."
"“Your Prius buyers and Tesla buyers are not the same group,” he said. “The Tesla buyers are rich white males. They’re really about showing off a great sports car. So Elon is not concerned at all that people will stop buying his vehicles because he’s making fun of a Democratic president.”"
"Omg, anxiety cure...lol. Unfortunately the very desperate get caught up in these types of things and it's sad that we live in a world where people take advantage of that...kinda makes you anxious right?!? The root of anxiety is trust and when people abuse that it forms part of the vicious cycle. Learning to trust in a corrupt world is hard enough...add anxiety disorder to the equation...disaster!!!"
"I don't want you to feel ashamed to have BPD. I will say there's so much stigma attached to it, personally and professionally. People hear personality disorder and really don't understand what that means. I don't tell people anymore, or not right away at least, because I want to be seen as a person, not a disorder."
"Creeping Socialism, two steps forward, one step back, and they are making progress. Humans adapt rapidly but someday we are going to wake up to the government micro-managing our lives and it will be too late to undo all the legislation run by the largest untrained bureaucracy the world has ever known."
"Some anon asked if anyone had gotten into possession for any other reason aside from it turning you on like if your trans for example some trans people responded and got insulted, then some other anon came along and basically asked “what the hell is your guy’s problem?” And it went on from there, just go see the thread"
"“We were the base, the first base that were taking the Afghanistan refugees… I am sorry… seeing all the kids with their families and they didn’t know how the rest of the day or even the rest of their lives would go. Just knowing that I had my kids at home safe and I would see them and they would have a roof over their heads and food to eat, it made me feel very blessed.”"
"The difference between a 'true' mind control like 'I just push a button and she's mine forever' and applied mind control is a matter of technique over time. I have no doubt that there are stage hypnotists and erotic hypnotists having zero problem getting people to do what they want, but that's a skill they've cultivated."
"In Fulton County, Georgia, two election office workers — one a temporary employee — filed a lawsuit in December against a conservative website, accusing it of spreading false stories about them. Their lawsuit said the false claims led to a “deluge of intimidation, harassment, and threats that has forced them to change their phone numbers, delete their online accounts, and fear for their physical safety.”"


FNs
"The 2019 crime rate in Atascadero, CA is 137 (City-Data.com crime index), which is 2.0 times lower than the U.S. average. It was higher than in 53.5% U.S. cities. The 2019 Atascadero crime rate fell by 16% compared to 2018. In the last 5 years Atascadero has seen decline of violent crime and decline of property crime."
"The first point of call should be your GP as these symptomes can be related to depleted iron, vitamins or perhaps a thyroid disorder and many more besides. The cycling nature of the mood suggests a disorder similar to bi-polar but as you will hear form other forumites you need a clinical assesment and formal diagnosis. Good luck to you though."
"I'm only say this because I think I understand how you are explaining a hypomania, I have on occasions taken some speed in the past & people have said I dont stop talking & i know you can just keep going n going like once I just kept cleaning I just couldnt sit down for about three days, The only reason why I've said this is because I thought that I didnt understand, but I can relate though under normal circumstances I only really understand the lows because I dont have highs."
"Sorry fag, if you haven't noticed we only post VNs with still images here. Actual animations? Fuck outta here. Also that dev is supposedly making a strategy RPG to go along with his game eventually? Fucking blasphemy. Why the fuck doesn't he just shit out terribly dated still renders of daz3d models with some cringe text and call it a day. That's the real spice."
"Its because umemaro doesn't make stuff for you or us non jap viewers. Ntr is a big thing to exploit for them and sells good. Also if you havent figured it out this is a prequel of the previous one. Apart from giving the sex situations in his animations, umemaro likes to put story into them. Every girl of him in the umeverse have a background and he likes to share that. In this case how the sexually frustrated Mari by her always traveling business husband plans how to satisfy her need by any means. Wouldnt be surprised if another title gets in the middle of this 2 sexual circumstances works"
"Inflated egos one, two and three on Google drive. And e-hentai. Third part. Same shit, but with more artists. Art and stories boring, terrible and banal. There are even 3 images and stories from the same artist. From the same fucking artist. This is genuinely a mockery both for those who buy it and for the inflation community."
"today he has not replied to any of my texts. im trying to stay calm and not panic and just carrying on as normal, telling he im there for him and wont leave him. hes pushing me away, and part of me thinks i cant cope and should leave but a bigger part of me says i love him and want to stay and help him through"
"The Kojima fanbase even entertains thay idea. The ""strand genre"" thing was never brought up in ads or anything, it was just dumb shit that he said in an interview and fans ate it up. There are literally videos on youtube trying to break down what the ""strand genre"" is and trying to argue that Kojima isn't just taking credit for what Dark Souls did."
"Oh, they definitely need to slow the fuck down on all these fanfic/additional scenes they keep doing since it's draining way too much dev time. They really need to start thinking about the tents and miscellaneous scenes, they already added quite a few but a return of barmaid/whoring is really, really needed."
"Depends. If they're actually drawn on-model, they're monstrous enough to belong here. The problem is furries flock to them like flies on shit and drawn them with cute cartoon proportions or big bara muscle-bodies. Or waifufags churning out garbage R63 Skaven women. Skavanon is one of the few old drawfags who actually made them as fucked-up and nasty as they're supposed to be, and he vanished into the aether years ago."
